In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from local_plot import *
from utils import *
import re


In [3]:
def extract_broadcast_D2VINS_from_log(log_content):
    # Sample log:
    # [D2VINS] Broadcast VINS Data size 4144 with 54 poses 10 extrinsic.
    # Match using regex, to match broadcast poses and extrinsic
    pattern = re.compile(r"\[D2VINS\] Broadcast VINS Data size (\d+) with (\d+) poses (\d+) extrinsic.")
    matched = pattern.findall(log_content)
    # Find the last line of log_content contain broadcast poses
    lines = log_content.splitlines()
    c = len(lines)
    for line in reversed(lines):
        matched = pattern.findall(line)
        c -= 1
        if matched:
            break
    matched = np.array(matched, dtype=float)
    return np.sum(matched[:,0]), np.sum(matched[:, 1]) + np.sum(matched[:, 2]), c

def extract_broadcast_D2PGO_from_log(log_content):
    # [Drone 2] DPGO broadcast poses 249
    # Match using regex, to match broadcast poses
    pattern = re.compile(r"\[Drone \d+\] DPGO broadcast poses (\d+)")
    matched = pattern.findall(log_content)
    matched = np.array(matched, dtype=float)
    return np.sum(matched)

def extract_broadcast_poses(log_content):
    broadcast_D2VINS, sum_poses, line_no = extract_broadcast_D2VINS_from_log(log_content)
    broadcast_D2PGO = extract_broadcast_D2PGO_from_log(log_content)
    return broadcast_D2VINS + broadcast_D2PGO*2, sum_poses

def extract_D2Comm_broadcast_size(log_content):
    #[D2Comm] Broadcast PGO data of drone 1, lcm 27576 bytes. 
    # Match using regex, to match broadcast poses
    pattern = re.compile(r"\[D2Comm\] Broadcast PGO data of drone \d+, lcm (\d+) bytes.")
    matched = pattern.findall(log_content)
    matched = np.array(matched, dtype=float)
    return np.sum(matched)

def extract_frontend_sum_kb(log_content):
    # [SWARM_LOOP](5584) BD KF 1005889@1 LM: 76 size 10211 header 311 avgsize 20120 sumkB 109713 avgLM 137 need_send_features: 1
    # Match using regex, to match last sumkB
    pattern = re.compile(r"\[SWARM_LOOP\]\(\d+\) BD KF \d+@\d+ LM: \d+ size \d+ header \d+ avgsize \d+ sumkB (\d+) avgLM \d+ need_send_features: \d+")
    matched = pattern.findall(log_content)
    matched = np.array(matched, dtype=float)
    return matched[-1]
    
    
def process_bd_for_backend(path, nodes=range(1, 6)):
    broadcast_poses = 0
    frontend_sum_kb = 0
    d2vins_size = 0
    d2pgo_size = 0
    for i in nodes:
        log_vo = f"{path}{i}/d2slam.log"
        with open(log_vo, "r") as f:
            log_content = f.read()
            _d2vins_size, _broadcast_poses = extract_broadcast_poses(log_content)
            d2vins_size += _d2vins_size
            broadcast_poses += _broadcast_poses
            frontend_sum_kb += extract_frontend_sum_kb(log_content)
            d2pgo_size += extract_D2Comm_broadcast_size(log_content)

    print(f"Total broadcast poses {broadcast_poses/1024/1024:.1f}")
    print(f"Total frontend sum {frontend_sum_kb/1024:.1f} MB")
    print(f"Total d2vins size {d2vins_size/1024/1024:.1f} MB")
    print(f"Total d2pg size {d2pgo_size/1024/1024:.1f} MB")


In [8]:
path = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2slam/swarm"
process_bd_for_backend(path)


Last number of broadcast poses 41687


IndexError: index -1 is out of bounds for axis 0 with size 0

In [4]:
path = "/home/xuhao/data/d2slam/quadcam_7inch_n3_2023_1_14/outputs/d2slam-5-yaw/swarm"
process_bd_for_backend(path)

Total broadcast poses 0.0
Total frontend sum 1546.7 MB
Total d2vins size 75.9 MB
Total d2pg size 2340.9 MB


In [5]:
path = "/home/xuhao/data/d2slam/quadcam_7inch_n3_2023_1_14/outputs/d2slam-5-yaw-compact/swarm"
process_bd_for_backend(path)

Total broadcast poses 0.0
Total frontend sum 984.2 MB
Total d2vins size 70.8 MB
Total d2pg size 2179.4 MB


In [12]:
path = "/home/xuhao/data/d2slam/tum_corr/outputs/d2slam/swarm"
process_bd_for_backend(path)

Total broadcast poses 0.0
Total frontend sum 344.2 MB
Total d2vins size 2.4 MB
Total d2pg size 60.1 MB


In [13]:
path = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2slam/swarm"
process_bd_for_backend(path)

Total broadcast poses 0.0
Total frontend sum 266.4 MB
Total d2vins size 1.6 MB
Total d2pg size 42.0 MB


In [12]:
path = "/home/xuhao/data/d2slam/tum_datasets/outputs/d2slam-compact/swarm"
process_bd_for_backend(path)

Total broadcast poses 0.0
Total frontend sum 244.2 MB
Total d2vins size 1.2 MB
Total d2pg size 31.0 MB


In [10]:
path = "/home/xuhao/data/d2slam/tum_corr/outputs/d2slam_compact/swarm"
process_bd_for_backend(path)

Total broadcast poses 0.0
Total frontend sum 323.0 MB
Total d2vins size 2.9 MB
Total d2pg size 74.1 MB


In [3]:
path = "/home/xuhao/data/d2slam/ri_realsense_walkaround_2022_10/outputs/fuse_all-3/swarm"
process_bd_for_backend(path, [1, 2, 3])

Total broadcast poses 0.7
Total frontend sum 231.8 MB
Total d2vins size 10.1 MB
Total d2pg size 13.4 MB


In [3]:
path = "/home/xuhao/data/d2slam/ri_realsense_walkaround_2022_10/outputs/d2slam_compact/swarm"
process_bd_for_backend(path, [1, 2, 3])

Total broadcast poses 0.0
Total frontend sum 105.4 MB
Total d2vins size 0.4 MB
Total d2pg size 11.9 MB
